In [91]:
import pandas as pd
import re
import json

# Define an empty list to store dictionaries representing each item
items_list = {}
i=1
items_list[1]=[]
with open('data.txt', 'r') as file:
    for line in file:
        line = line.strip()# Remove leading/trailing whitespaces
        if not line:  # If line is empty, it indicates a new item
            i+=1
            items_list[i] = []
        else:
            items_list[i].append(line)

In [89]:
# items_list

In [44]:
book=['Id:   1',
  'ASIN: 0827229534',
  'title: Patterns of Preaching: A Sermon Sampler',
  'group: Book',
  'salesrank: 396585',
  'similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
  'categories: 2',
  '|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
  '|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
  'reviews: total: 2  downloaded: 2  avg rating: 5',
  '2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
  '2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5']

In [92]:
pattern = r'^[a-zA-Z]+:.*$'
date_rx = r'^\d{4}-(?:\d{1}|\d{2})-(?:\d{1}|\d{2})'
pattern1 = r'(\w+): (\d+)'
new_list={}
id=0
for Id, book in items_list.items():
    dict={}
    for line in book:
        if re.match(pattern, line):
            key, value = line.split(":", 1)
            value = value.strip()
            # print("key-",key," value-",value)
            dict[key]=value
            if key=="Id":
                id=int(value)
            if key=='categories':
                dict[key]=[]
            if key=='similar':
                arr=line.split()
                dict['similar']=arr[1:].copy()
            if key=="reviews":
                dict['reviews']={}
                matches = re.findall(pattern1, value)
                for match in matches:
                    k_, v_ = match
                    dict['reviews'][k_]=v_
                dict['reviews']['individual_reviews']=[]
                
        elif re.match(date_rx, line):
            dct={}
            arr=line.split()
            dct['date']=arr[0]
            dct['customer']=arr[2]
            dct['rating']=int(arr[4])
            dct['votes']=int(arr[6])
            dct['helpful']=int(arr[8])
            dict['reviews']['individual_reviews'].append(dct)
        else:
            dict['categories'].append(line)
            
    new_list[id]=dict
    # else:
    #     print(f"Not matched: {line}")
 

In [93]:
new_list[1]

{'Id': '1',
 'ASIN': '0827229534',
 'title': 'Patterns of Preaching: A Sermon Sampler',
 'group': 'Book',
 'salesrank': '396585',
 'similar': ['5',
  '0804215715',
  '156101074X',
  '0687023955',
  '0687074231',
  '082721619X'],
 'categories': ['|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
  '|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]'],
 'reviews': {'total': '2',
  'downloaded': '2',
  'rating': '5',
  'individual_reviews': [{'date': '2000-7-28',
    'customer': 'A2JW67OY8U6HHK',
    'rating': 5,
    'votes': 10,
    'helpful': 9},
   {'date': '2003-12-14',
    'customer': 'A2VE83MZF98ITY',
    'rating': 5,
    'votes': 6,
    'helpful': 5}]}}

In [69]:
date_rx = r'^\d{4}-(?:\d{1}|\d{2})-(?:\d{1}|\d{2})'
reviews=['2002-1-7  cutomer:  A9CQ3PLRNIR83  rating: 4  votes:   5  helpful:   5',
   '2002-1-24  cutomer: A13SG9ACZ9O5IM  rating: 5  votes:   8  helpful:   8',
   '2002-1-28  cutomer: A1BDAI6VEYMAZA  rating: 5  votes:   4  helpful:   4',
   '2002-2-6  cutomer: A2P6KAWXJ16234  rating: 4  votes:  16  helpful:  16',
   '2002-2-14  cutomer:  AMACWC3M7PQFR  rating: 4  votes:   5  helpful:   5',
   '2002-3-23  cutomer: A3GO7UV9XX14D8  rating: 4  votes:   6  helpful:   6']
# rx = re.compile(date_rx, re.I)
review_list=[]
for item in reviews:
    # match = rx.findall(item)
    dct={}
    arr=item.split()
    dct['date']=arr[0]
    dct['customer']=arr[2]
    dct['rating']=int(arr[4])
    dct['votes']=int(arr[6])
    dct['helpful']=int(arr[8])
    review_list.append(dct)

In [85]:
similar= '5  0738700827  1567184960  1567182836  0738700525  0738700940'
arr=similar.split()
arr=arr[1:]
print(arr[0])

0738700827


In [95]:
file_path = "data.json"
with open(file_path, "w") as json_file:
    json.dump(new_list, json_file)